In [1]:
import requests
import os

def download_file(url, filename=None):
  """Downloads a file from a URL and saves it locally.

  Args:
    url: The URL of the file to download.
    filename: The filename to save the downloaded file as. If not specified,
      the filename will be extracted from the URL.
  """
  response = requests.get(url, stream=True)
  response.raise_for_status()
  
  if filename is None:
    filename = url.split("/")[-1]

  if os.path.isfile(filename):
    os.remove(filename)  

  if os.path.isfile(filename+".wal"):
    os.remove(filename+".wal")  

  with open(filename, "wb") as f:
    for chunk in response.iter_content(1024):
      if chunk:  # filter out keep-alive new chunks
        f.write(chunk)

  return filename

In [2]:
for x in os.listdir():
    if x.endswith(".db") or x.endswith(".wal"):
        # Prints only text file present in My Folder
        print(x)

dev.db
dev.db.wal


Download Schema

In [3]:
url = "http://dev-public-storage-440955376164.s3-website.us-east-2.amazonaws.com/catalogs/dev.db"
downloaded_filename = download_file(url)
downloaded_filename

'dev.db'

Open Database and Login

In [4]:
import duckdb
from duckdb import login

connection = duckdb.login.cognito("vaultdb","test123", downloaded_filename)

ModuleNotFoundError: No module named 'duckdb'

In [7]:
connection.execute("SELECT * FROM vaultdb_configs").fetchdf()

,database_name,database_oid,schema_name,schema_oid,config_name,config_oid,internal,temporary,config_value
0,dev,1096,security,1100,application_name,1111,False,False,dev
1,dev,1096,security,1100,identity_pool_id,1113,False,False,us-east-2:9b07f85f-bb93-49d5-bdea-7921cf7b247a
2,dev,1096,security,1100,remote,1115,False,False,s3://dev-data-440955376164/dev
3,dev,1096,security,1100,remote_merge_path,1117,False,False,s3://dev-public-storage-440955376164/dev
4,dev,1096,security,1100,user_pool_client_id,1119,False,False,i9iogmrgf9qu3lrqsdlv5ul65
5,dev,1096,security,1100,user_pool_id,1121,False,False,us-east-2_0G3LUmwQn


Downloaded Database is empty

In [8]:
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,1,2,3,4,5,6,7,8
1,11,22,33,44,55,66,77,88
2,111,222,333,444,555,666,777,888
3,1111,2222,3333,4444,5555,6666,7777,8888


Insert Data Locally

In [9]:
connection.execute("INSERT INTO another_T VALUES (91,92,93,94,95,96,97,98)")

In [10]:
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,1,2,3,4,5,6,7,8
1,11,22,33,44,55,66,77,88
2,111,222,333,444,555,666,777,888
3,1111,2222,3333,4444,5555,6666,7777,8888
4,91,92,93,94,95,96,97,98


Data Inheritence

In [11]:
connection.execute(f"PRAGMA enable_data_inheritance;")
connection.execute("SELECT * FROM another_T").fetchdf()

,col1,col2,col3,col4,col5,col6,col7,col8
0,1,2,3,4,5,6,7,8
1,11,22,33,44,55,66,77,88
2,111,222,333,444,555,666,777,888
3,1111,2222,3333,4444,5555,6666,7777,8888
4,91,92,93,94,95,96,97,98


Create Objects Locally

In [12]:
connection.execute('BEGIN TRANSACTION;')
connection.execute('CREATE TABLE demo (col1 INT, col2 INT, col3 INT, col4 INT, col5 INT, col6 INT, col7 INT, col8 INT)')
connection.execute("INSERT INTO demo VALUES (1,2,3,4,5,6,7,8), (11,22,33,44,55,66,77,88), (111,222,333,444,555,666,777,888), (1111,2222,3333,4444,5555,6666,7777,8888)")
connection.execute('COMMIT;')

In [13]:
connection.execute('PUSH DATABASE dev;').fetchdf()

,Success


In [3]:
connection.execute('SELECT * FROM duckdb_secrets();').fetchdf()

,name,type,provider,persistent,storage,scope,secret_string
0,vaultdb,s3,config,False,memory,"[s3://, s3n://, s3a://]",name=vaultdb;type=s3;provider=config;serializa...
